In [ ]:
from concurrent.futures import ProcessPoolExecutor
from pathlib import Path
from natsort import natsorted
import pandas as pd

In [ ]:
import sys, os

path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir, 'TestBeam')))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

from beamtest_analysis_helper import DecodeBinary

In [ ]:
def decode_directory(path_to_dir):
    # Assuming each directory has a list of files to decode
    files = sorted(list(path_to_dir.glob('TDC*bin')))
    decoder = DecodeBinary(
        firmware_key=0b0001,
        board_id=[0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f],
        file_list=files,
        save_nem=None,
        skip_fw_filler=True,
        skip_event_df=True,
        skip_crc_df=True,
    )
    df, _, _, filler_df = decoder.decode_files()

    return df, filler_df

In [ ]:
dirs = natsorted(list(Path('./CERN_Aug_2024_TID/ET2p03_BARE9_1p20VA_1p20VD_200MRad_23h00m00s_m23p6').glob('ET2p03_BARE9*Threshold*')))
dirs[:5]

In [ ]:
with ProcessPoolExecutor() as executor:
    results = list(executor.map(decode_directory, dirs))

In [ ]:
dfs = []

for idx, idir in enumerate(dirs):
    info = idir.name.split('_')
    thres = int(info[-1])
    charge = int(info[-3])

    tmp = results[idx][0][1000:].copy()

    if tmp.shape[0] < 100:
        continue

    tmp.loc[:, 'charge'] = charge
    tmp.loc[:, 'threshold'] = thres

    tmp['charge'] = tmp['charge'].astype('uint8')
    tmp['threshold'] = tmp['threshold'].astype('uint16')

    dfs.append(tmp)

df = pd.concat(dfs)
del dfs

df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_feather('ET2p03_BARE9_1p2VD_1p20VA_m23p7C_200MRad_23h_qinj_moneyplot.feather')

In [ ]:
### filler

filler_dfs = []
for idx, idir in enumerate(dirs):
    info = idir.name.split('_')
    thres = int(info[-1])
    charge = int(info[-3])

    tmp = results[idx][1].copy()

    if tmp.empty:
        continue

    tmp.loc[:, 'charge'] = charge
    tmp.loc[:, 'threshold'] = thres

    tmp['charge'] = tmp['charge'].astype('uint8')
    tmp['threshold'] = tmp['threshold'].astype('uint16')

    filler_dfs.append(tmp)

filler_df = pd.concat(filler_dfs)
del filler_dfs

filler_df.reset_index(drop=True, inplace=True)

In [ ]:
filler_df.to_feather('ET2p03_BARE9_1p2VD_1p20VA_m23p7C_200MRad_23h_qinj_moneyplot_filler.feather')